In [2]:
%cd ~/ChestXray-14/

/home/jovyan/ChestXray-14


In [3]:
"""
Chonsawat Path: input_path = "/content/drive/MyDrive/KKU /Project/Dataset/ChestXray NIH"
Deepnote Path: input_path = "/datasets/chonsawat-drive/KKU /Project/Dataset/ChestXray NIH"
Elab Path: input_path = "~/ChestXray-14/dataset/ChestXray NIH"
"""
input_path = "dataset/ChestXray NIH"

from sklearn.utils import shuffle
from tqdm.notebook import tqdm
import tensorflow as tf
import pandas as pd
import numpy as np
import os

import warnings
warnings.filterwarnings('ignore')

In [4]:
STRATEGY = tf.distribute.get_strategy()    
BATCH_SIZE = 16
IMG_SIZE = 224
SEED = 42
    
print('Using tensorflow %s' % tf.__version__)

Using tensorflow 2.8.1


In [5]:
feature_map = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'image_id': tf.io.FixedLenFeature([], tf.string),
    'No Finding': tf.io.FixedLenFeature([], tf.int64),
    'Atelectasis': tf.io.FixedLenFeature([], tf.int64),
    'Consolidation': tf.io.FixedLenFeature([], tf.int64),
    'Infiltration': tf.io.FixedLenFeature([], tf.int64),
    'Pneumothorax': tf.io.FixedLenFeature([], tf.int64),
    'Edema': tf.io.FixedLenFeature([], tf.int64),
    'Emphysema': tf.io.FixedLenFeature([], tf.int64),
    'Fibrosis': tf.io.FixedLenFeature([], tf.int64),
    'Effusion': tf.io.FixedLenFeature([], tf.int64),
    'Pneumonia': tf.io.FixedLenFeature([], tf.int64),
    'Pleural_Thickening': tf.io.FixedLenFeature([], tf.int64),
    'Cardiomegaly': tf.io.FixedLenFeature([], tf.int64),
    'Nodule': tf.io.FixedLenFeature([], tf.int64),
    'Mass': tf.io.FixedLenFeature([], tf.int64),
    'Hernia': tf.io.FixedLenFeature([], tf.int64)}


def count_data_items(filenames):
    return np.sum([int(x[:-6].split('-')[-1]) for x in filenames])


def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=1)
    image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, 1])
    return image


def scale_image(image, target):
    image = tf.cast(image, tf.float32) / 255.
    return image, target


def read_tfrecord(example):
    example = tf.io.parse_single_example(example, feature_map)
    image = decode_image(example['image'])
    target = [
        example['No Finding'],
        example['Atelectasis'],
        example['Consolidation'],
        example['Infiltration'],
        example['Pneumothorax'],
        example['Edema'],
        example['Emphysema'],
        example['Fibrosis'],
        example['Effusion'],
        example['Pneumonia'],
        example['Pleural_Thickening'],
        example['Cardiomegaly'],
        example['Nodule'],
        example['Mass'],
        example['Hernia']]
    return image, target


def data_augment(image, target):
    image = tf.image.random_flip_left_right(image, seed=SEED)
    image = tf.image.random_flip_up_down(image, seed=SEED)
    return image, target


def get_dataset(filenames, shuffled=False, repeated=False, 
                cached=False, augmented=False, distributed=True):
    auto = tf.data.experimental.AUTOTUNE
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=auto)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=auto)
    if augmented:
        dataset = dataset.map(data_augment, num_parallel_calls=auto)
    dataset = dataset.map(scale_image, num_parallel_calls=auto)
    if shuffled:
        dataset = dataset.shuffle(2048, seed=SEED)
    if repeated:
        dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    if cached:
        dataset = dataset.cache()
    dataset = dataset.prefetch(auto)
    if distributed:
        dataset = STRATEGY.experimental_distribute_dataset(dataset)
    return dataset


def get_model():
    model = tf.keras.models.Sequential([
        tf.keras.applications.EfficientNetB0(
            include_top=False,
            input_shape=(None, None, 1),
            weights=None,
            pooling='avg'),
        tf.keras.layers.Dense(15, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=tf.keras.metrics.AUC(multi_label=True))

    return model

def get_resnet50_model():
    model = tf.keras.models.Sequential([
        tf.keras.applications.resnet50.ResNet50(
            include_top=False, 
            input_shape=(None, None, 1), 
            weights=None,
            pooling='avg'
            # classes=1000,
        ),
        tf.keras.layers.Dense(15, activation='sigmoid')
    ])
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        run_eagerly=True,
        metrics=tf.keras.metrics.AUC(multi_label=True))

    return model

In [6]:
train_filenames = tf.io.gfile.glob(f'{input_path}/data/224x224/train/*.tfrec')
val_filenames = tf.io.gfile.glob(f'{input_path}/data/224x224/valid/*.tfrec')
test_filenames = tf.io.gfile.glob(f'{input_path}/data/224x224/test/*.tfrec')

steps_per_epoch = count_data_items(train_filenames) // BATCH_SIZE
validation_steps = count_data_items(val_filenames) // BATCH_SIZE

train_dataset = get_dataset(train_filenames, shuffled=True, repeated=True, augmented=True)
val_dataset = get_dataset(val_filenames, cached=True)

with STRATEGY.scope():
    model = get_resnet50_model()
    
history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=5,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    verbose=1)

2022-06-13 07:10:59.401505: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
4905/4905 [==============================] - 20404s 4s/step - loss: 0.2190 - auc: 0.5674 - val_loss: 0.2828 - val_auc: 0.5809
Epoch 2/5
4905/4905 [==============================] - 20095s 4s/step - loss: 0.2112 - auc: 0.6127 - val_loss: 0.2121 - val_auc: 0.6345
Epoch 3/5
4905/4905 [==============================] - 19948s 4s/step - loss: 0.2079 - auc: 0.6409 - val_loss: 0.2076 - val_auc: 0.6533
Epoch 4/5
4905/4905 [==============================] - 19916s 4s/step - loss: 0.2052 - auc: 0.6609 - val_loss: 0.2081 - val_auc: 0.6748
Epoch 5/5
4905/4905 [==============================] - 19945s 4s/step - loss: 0.2030 - auc: 0.6764 - val_loss: 0.2009 - val_auc: 0.6910


In [7]:
model.save(f"{input_path}/models/Resnet50.h5")